# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder, StandardScaler

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
# Understanding the dimension and the statistics of the dataframe
display(df.shape)
display(df.describe())

(68817, 86)

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [6]:
# Understand the column names as a list
column_values = df.columns.values.tolist()
print(column_values)

['loan_amnt', 'int_rate', 'installment', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'next_pymnt_d', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'm

In [7]:
# Dropping columns with dates. The assumption here is that the dates by themselves do not indicate the riskiness of a loan, unless a certain calculation is performed with the time series evaluated as a consideration.
df.drop(columns=['issue_d', 'next_pymnt_d'], inplace=True)

In [8]:
# Creating a dataframe with columns that have string values for further evaluation
strings_df = df[['home_ownership','verification_status','loan_status','pymnt_plan','initial_list_status','application_type','hardship_flag','debt_settlement_flag']].copy()

In [9]:
# Creating a new dataframe with the value_counts of string values
strings_df_ct = strings_df.apply(pd.Series.value_counts)
strings_df_ct

,home_ownership,verification_status,loan_status,pymnt_plan,initial_list_status,application_type,hardship_flag,debt_settlement_flag
ANY,639.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Individual,NaN,NaN,NaN,NaN,NaN,59206.0,NaN,NaN
Joint App,NaN,NaN,NaN,NaN,NaN,9611.0,NaN,NaN
MORTGAGE,36219.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
N,NaN,NaN,NaN,NaN,NaN,NaN,68817.0,68817.0
Not Verified,NaN,32895.0,NaN,NaN,NaN,NaN,NaN,NaN
OWN,7346.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RENT,24613.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Source Verified,NaN,25737.0,NaN,NaN,NaN,NaN,NaN,NaN
Verified,NaN,10185.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Based on the value counts of each column, pymnt_plan, hardship_flag and debt_settlement_flag do not seem to have a distinction in different values. 
# Dropping the "flag" columns
df.drop(columns=['pymnt_plan','hardship_flag','debt_settlement_flag'], inplace=True)

# Creating an instance of label encoder
le = LabelEncoder()

In [11]:
# Fitting and encoding columns with label encoder

# home_ownership column
le.fit(df['home_ownership'])
df['home_ownership'] = le.transform(df['home_ownership'])

# verification_status column
le.fit(df['verification_status'])
df['verification_status'] = le.transform(df['verification_status'])

# loan_status column
le.fit(df['loan_status'])
df['loan_status'] = le.transform(df['loan_status'])

# initial_list_status column
le.fit(df['initial_list_status'])
df['initial_list_status'] = le.transform(df['initial_list_status'])

# application_type column
le.fit(df['application_type'])
df['application_type'] = le.transform(df['application_type'])

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,dti,delinq_2yrs,inq_last_6mths,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,10500.0,0.1719,375.35,3,66000.0,1,1,27.24,0.0,0.0,...,0.0,3.0,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0
1,25000.0,0.2000,929.09,1,105000.0,2,1,20.23,0.0,0.0,...,0.0,6.0,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0
2,20000.0,0.2000,529.88,1,56000.0,2,1,24.26,0.0,0.0,...,0.0,4.0,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0
3,10000.0,0.1640,353.55,3,92000.0,2,1,31.44,0.0,1.0,...,0.0,1.0,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0
4,22000.0,0.1474,520.39,1,52000.0,0,1,18.76,0.0,1.0,...,0.0,1.0,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0


## Split the Data into Training and Testing

In [12]:
# Create our features
X = df.drop(columns='loan_status')

# Create our target
y = df['loan_status']

In [13]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,21.778153,0.217766,0.497697,12.587340,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,20.199244,0.718367,0.758122,6.022869,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.000000,0.000000,2.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,13.890000,0.000000,0.000000,8.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,19.760000,0.000000,0.000000,11.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,26.660000,0.000000,1.000000,16.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,999.000000,18.000000,5.000000,72.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [16]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [18]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [20]:
# Create the StandardScaler instance
scl = StandardScaler()  

In [21]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scl.fit(X_train)

In [ ]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
Counter(y_train)

Counter({1: 51366, 0: 246})

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the following steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [50]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf_classify = BalancedRandomForestClassifier(n_estimators=600, max_depth=10, random_state=1)
brf_classify.fit(X,y)

BalancedRandomForestClassifier(max_depth=10, n_estimators=600, random_state=1)

In [51]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_true = y_test
y_pred = brf_classify.predict(X_test)
balanced_accuracy_score(y_true, y_pred)

0.9365341556372662

In [52]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   98,     3],
       [ 1663, 15441]], dtype=int64)

In [53]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.06      0.97      0.90      0.11      0.94      0.88       101
          1       1.00      0.90      0.97      0.95      0.94      0.87     17104

avg / total       0.99      0.90      0.97      0.94      0.94      0.87     17205



In [61]:
# List the features sorted in descending order by feature importance
importances = brf_classify.feature_importances_

In [64]:
z = zip(importances, X.columns)
sorted(z, reverse=True)

[(0.08799274866419383, 'total_rec_prncp'),
 (0.07069207808152127, 'last_pymnt_amnt'),
 (0.06936331742213929, 'total_pymnt'),
 (0.06719302093852682, 'total_pymnt_inv'),
 (0.0656450970089439, 'total_rec_int'),
 (0.03132278766469822, 'int_rate'),
 (0.018256892460896193, 'dti'),
 (0.017398797974953126, 'max_bal_bc'),
 (0.01673116319741864, 'installment'),
 (0.016677270510253105, 'annual_inc'),
 (0.016311728510687795, 'out_prncp'),
 (0.016052262053771874, 'mths_since_recent_inq'),
 (0.015456171439843246, 'bc_util'),
 (0.015118414931391821, 'il_util'),
 (0.014915067269174225, 'mo_sin_old_rev_tl_op'),
 (0.014800380291571881, 'avg_cur_bal'),
 (0.014759809331322007, 'out_prncp_inv'),
 (0.01458551365153881, 'total_bc_limit'),
 (0.013952901710041316, 'revol_bal'),
 (0.013704413063686059, 'bc_open_to_buy'),
 (0.013550092232194257, 'tot_hi_cred_lim'),
 (0.013497341799269851, 'tot_cur_bal'),
 (0.01349450698782415, 'all_util'),
 (0.013327240275029219, 'mths_since_rcnt_il'),
 (0.013152813663399562, 'm

### Easy Ensemble Classifier

In [65]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
easy = EasyEnsembleClassifier(n_estimators=600, random_state=1)
easy.fit(X,y)

EasyEnsembleClassifier(n_estimators=600, random_state=1)

In [69]:
# Calculated the balanced accuracy score
y_true = y_test
y_pred = easy.predict(X_test)
balanced_accuracy_score(y_true, y_pred)

0.8351508882179144

In [70]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   89,    12],
       [ 3607, 13497]], dtype=int64)

In [71]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.02      0.88      0.79      0.05      0.83      0.70       101
          1       1.00      0.79      0.88      0.88      0.83      0.69     17104

avg / total       0.99      0.79      0.88      0.88      0.83      0.69     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    Balanced Random Forest Classifier

2. Which model had the best recall score?

    Balanced Random Forest Classifier

3. Which model had the best geometric mean score?

    Balanced Random Forest Classifier

4. What are the top three features?

    total_rec_prncp, last_pymnt_amnt, total_pymnt